In [1]:
import numpy as np

In [106]:
# Формула расчета количества элементов по оси после свертки:
#
#   N = (N_in - N_kernel + 2 * N_pad) / N_stride + 1, где:
#
# * N_in - число "пикселей" входного изображения по оси,
# * N_kernel - число элементов ядра по оси,
# * N_pad - размер паддинга по оси (учитвая, что до и после "настоящих" элементов его размер одинаков),
# * N_stride - величина шага по оси.
#
# Однако здесь мы считаем, что в матрице нет паддинга,
# так как ее можно просто изменить перед началом свертки.
def get_result_dimension(matrix_dim, kernel_dim, stride_dim):
    return (matrix_dim - kernel_dim) // stride_dim + 1


# Преобразовывает несколько участков матриц в одну матрицу, где исходные части являются вектор-столбцами.
def get_p(matrix_slices):
    return np.concatenate([matrix_slice.reshape(matrix_slice.size, 1) for matrix_slice in matrix_slices], axis=1)


# Получает все части матриц, которые понадобятся для свертки, в виде вектор-столбцов.
#
# Параметры:
# * matrix - матрица для свертки;
# * kernel_dim - 3 элемента, указывающие на размерность ядра (каналы, высота, ширина);
# * result_dim - 3 элемента, указывающие на размерность результата (каналы, высота, ширина);
# * stride - 3 элемента, указывающие на размер шага (по каналу, высоте и ширине).
def get_submatrices(matrix, kernel_dim, result_dim, stride):
    result = []

    # Идем по строкам матрицы
    x = 0
    for i in range(result_dim[1]):
        # Идем по элементам строки
        y = 0
        for j in range(result_dim[2]):
            # Получаем "кубический срез" по всем каналам, учиытвая размерность ядра свертки
            result.append(matrix[:, x : x + kernel_dim[1], y : y + kernel_dim[2]])
            y += stride[2]
        x += stride[1]

    # Выпрямляем кусочки матрицы и выстраиваем их в вектор-столбцы
    return get_p(result)


# Производит операцию свертки над изображением с C каналами, размером H x W пикселей,
# используя ядро размера H_k x W_k.
#
# Параметры:
# * matrix - исходное изображение размерности (C, H, W);
# * kernel - ядро свертки размерности (C, H_k, W_k);
# * padding - величина паддинга - тупл с 3 элементами: величина паддинга по оси каналов, по высоте и ширине;
# * stride - величина шага - тупл с 3 элементами: величина шага по оси каналов, по высоте и ширине.
def convolve(matrix, kernel, padding, stride):

    # Дополняем матрицу нулями, если нужно
    padded_matrix = np.pad(matrix, [(padding[i], ) for i in range(matrix.shape[0])], mode='constant', constant_values=0)

    # Рассчитываем размерности новой исходной картинки, ядра и результирующей матрицы
    matrix_dim = padded_matrix.shape
    kernel_dim = kernel.shape
    result_dim = [get_result_dimension(matrix_dim[i], kernel_dim[i], stride[i]) for i in range(matrix.shape[0])]

    # "Спрямляем" ядро
    K = kernel.reshape(1, kernel.size)

    # Получаем "подматрицы" исходного изображения для проведения умножения в виде вектор-столбцов
    P = get_submatrices(padded_matrix, kernel_dim, result_dim, stride)

    # Умножаем
    result = np.matmul(K, P)

    # Зная размерность результата, собираем его из получившейся на предыдущем шаге матрицы
    return result.reshape(result_dim[0], result_dim[1], result_dim[2])

In [107]:
X = np.array([[[1, 2, 3, 4],
               [5, 6, 7, 8],
               [9, 10, 11, 12],
               [13, 14, 15, 16]],
              [[17, 18, 19, 20],
               [21, 22, 23, 24],
               [25, 26, 27, 28],
               [29, 30, 31, 32]],
              [[33, 34, 35, 36],
               [37, 38, 39, 40],
               [41, 42, 43, 44],
               [45, 46, 47, 48]]])

W = np.array([[[1, 2],
               [3, 4]],
              [[5, 6],
               [7, 8]],
              [[9, 10],
               [11, 12]]])

convolve(X, W, padding=(0, 0, 0), stride=(1, 1, 1))

array([[ 1,  2,  3,  5,  6,  7,  9, 10, 11],
       [ 2,  3,  4,  6,  7,  8, 10, 11, 12],
       [ 5,  6,  7,  9, 10, 11, 13, 14, 15],
       [ 6,  7,  8, 10, 11, 12, 14, 15, 16],
       [17, 18, 19, 21, 22, 23, 25, 26, 27],
       [18, 19, 20, 22, 23, 24, 26, 27, 28],
       [21, 22, 23, 25, 26, 27, 29, 30, 31],
       [22, 23, 24, 26, 27, 28, 30, 31, 32],
       [33, 34, 35, 37, 38, 39, 41, 42, 43],
       [34, 35, 36, 38, 39, 40, 42, 43, 44],
       [37, 38, 39, 41, 42, 43, 45, 46, 47],
       [38, 39, 40, 42, 43, 44, 46, 47, 48]])

array([[[2060, 2138, 2216],
        [2372, 2450, 2528],
        [2684, 2762, 2840]]])